In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dat
import matplotlib as mplt
import datetime as dt
%matplotlib inline
sns.set()

In [3]:
auctions = pd.read_csv("data/auctions.csv", dtype={'date': 'str',
                                                   'device_id': 'category', 'ref_type_id': 'category',
                                                   'source_id':'category'}, parse_dates = ['date'])
auctions.head(5)

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0


In [4]:
auctions['mean_hour']=auctions['date'].dt.hour
auctions['std_hour']=auctions['date'].dt.hour
auctions=auctions.sort_values(by='date')


In [6]:
serie=auctions.groupby('device_id').cumcount()
serie=serie.to_list()

In [7]:
auctions['entradas_previas']=serie
auctions.head()

,date,device_id,ref_type_id,source_id,mean_hour,std_hour,entradas_previas
10129864,2019-04-18 00:00:00.015050,1826643666390887030,7,0,0,0,0
25285906,2019-04-18 00:00:00.029014,7037174172278258682,1,0,0,0,0
3946062,2019-04-18 00:00:00.057540,3392065368947589877,1,1,0,0,0
3946063,2019-04-18 00:00:00.126828,1228982273563226229,1,1,0,0,0
24607726,2019-04-18 00:00:00.132510,4123059034628125459,1,8,0,0,0


In [15]:
mean_hour_gb=auctions.groupby(['device_id'])['mean_hour'].expanding(2).mean()


In [20]:
mean_hour_gb=mean_hour_gb.reset_index()

In [27]:

mean_hour_gb=mean_hour_gb.set_index('level_1')
mean_hour_gb.head()

,level_0,index,device_id,mean_hour
level_1,,,,
17723834,0,0,100033926124811452,NaN
25608948,1,1,100033926124811452,3.0
15135224,2,2,100033926124811452,3.0
124213,3,3,100033926124811452,3.0
33738115,4,4,100033926124811452,3.0


In [29]:
mean_hour_gb=mean_hour_gb.drop(["level_0", "index", "device_id"], axis=1)
mean_hour_gb.head()

mean_hour    46934190
dtype: int64

In [30]:
auctions=auctions.merge(mean_hour_gb, how='left', left_index=True, right_index=True)

In [53]:
std_hour_gb=auctions.groupby('device_id')['std_hour'].expanding(2).std()

In [54]:
std_hour_gb=std_hour_gb.reset_index()

In [55]:
std_hour_gb=std_hour_gb.set_index('level_1')
std_hour_gb =std_hour_gb.drop(["device_id"], axis=1)

In [56]:
auctions=auctions.merge(std_hour_gb, how='left', left_index=True, right_index=True)

In [61]:
auctions.drop(['std_hour_x','mean_hour_x'],axis=1)
auctions.head()

,date,device_id_x,ref_type_id,source_id,mean_hour_x,std_hour_x,entradas_previas,mean_hour_y,device_id_y,std_hour_y
10129864,2019-04-18 00:00:00.015050,1826643666390887030,7,0,0,0,0,NaN,1826643666390887030,NaN
25285906,2019-04-18 00:00:00.029014,7037174172278258682,1,0,0,0,0,NaN,7037174172278258682,NaN
3946062,2019-04-18 00:00:00.057540,3392065368947589877,1,1,0,0,0,NaN,3392065368947589877,NaN
3946063,2019-04-18 00:00:00.126828,1228982273563226229,1,1,0,0,0,NaN,1228982273563226229,NaN
24607726,2019-04-18 00:00:00.132510,4123059034628125459,1,8,0,0,0,NaN,4123059034628125459,NaN


In [ ]:
export_csv = auctions.to_csv ('data/auctions_mean_std_calculados.csv', index = None, header=True)
#estos calculos tardan su tiempo, asi que mejor los  guardamos aparte

In [ ]:
fecha_minima=pd.to_datetime('2019-04-18 00:00:00')
fecha_minima
fecha_tope=pd.to_datetime('2019-04-21 00:00:00')
auctions_ventana1=auctions.loc[auctions['date']<fecha_tope].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana1['tiempo_hasta_aparicion']=(auctions_ventana1['date']-fecha_minima)/ np.timedelta64(24, 'h')

auctions_ventana1.head(25)
#auctions_ventana1['date'].max()

In [ ]:
auctions_ventana1_reducido=auctions_ventana1.groupby(['device_id']).agg({"date" : min,\
                                                                         "std_hour" : "first",\
                                                                         "mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana1_reducido=auctions_ventana1_reducido.dropna()

In [ ]:
fecha_minima=pd.to_datetime('2019-04-21 00:00:00')
fecha_tope=pd.to_datetime('2019-04-24 00:00:00')

auctions_ventana2=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana2['tiempo_hasta_aparicion']=(auctions_ventana2['date']-fecha_minima)/ np.timedelta64(24, 'h')

In [ ]:
auctions_ventana2.head()

In [ ]:
auctions_ventana2['dia_desvio_std']=auctions_ventana2['date'].dt.day
auctions_ventana2_reducido=auctions_ventana2.groupby(['device_id']).agg({"date" : min,\
                                                                         "std_hour" : "first",\
                                                                         "mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana2_reducido=auctions_ventana2_reducido.dropna()
auctions_ventana2_reducido.head()

In [ ]:
fecha_minima=pd.to_datetime('2019-04-22 00:00:00')
fecha_tope=pd.to_datetime('2019-04-25 00:00:00')

auctions_ventana3=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana3['tiempo_hasta_aparicion']=(auctions_ventana2['date']-fecha_minima)/ np.timedelta64(24, 'h')
auctions_ventana3['dia_mean']=auctions_ventana3['date'].dt.day
auctions_ventana3['dia_desvio_std']=auctions_ventana3['date'].dt.day
auctions_ventana3.head(25)
auctions_ventana3_reducido=auctions_ventana3.groupby(['device_id']).agg({"date" : min,\
                                                                         "std_hour" : "first",\
                                                                         "mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana3_reducido=auctions_ventana3_reducido.dropna()

In [ ]:
fecha_minima=pd.to_datetime('2019-04-23 00:00:00')
fecha_tope=pd.to_datetime('2019-04-26 00:00:00')

auctions_ventana4=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana4['tiempo_hasta_aparicion']=(auctions_ventana2['date']-fecha_minima)/ np.timedelta64(24, 'h')
auctions_ventana4['dia_mean']=auctions_ventana4['date'].dt.day
auctions_ventana4['dia_desvio_std']=auctions_ventana4['date'].dt.day
auctions_ventana4.head(25)
auctions_ventana4_reducido=auctions_ventana4.groupby(['device_id']).agg({"date" : min,\
                                                                         "std_hour" : "first",\
                                                                         "mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana4_reducido=auctions_ventana4_reducido.dropna()

In [ ]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00')
fecha_tope=pd.to_datetime('2019-04-27 00:00:00')

auctions_ventana5=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana5['tiempo_hasta_aparicion']=(auctions_ventana2['date']-fecha_minima)/ np.timedelta64(24, 'h')
auctions_ventana5['dia_mean']=auctions_ventana5['date'].dt.day
auctions_ventana5['dia_desvio_std']=auctions_ventana5['date'].dt.day
auctions_ventana5.head(25)
auctions_ventana5_reducido=auctions_ventana1.groupby(['device_id']).agg({"date" : min,\
                                                                         "std_hour" : "first",\
                                                                         "mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana5_reducido=auctions_ventana5_reducido.dropna()

In [ ]:
installs = pd.read_csv("data/installs.csv", dtype={'auction_type_id': 'float64', 'country': 'int64', 'date': 'str',
                                                   'device_id': 'int64', 'platform':'int64', 'ref_type_id': 'int64',
                                                   'source_id':'int64','ref_hash':'category' }, parse_dates = ['created'],low_memory=False)

installs.head(5)

In [ ]:
auctions_ventana1_con_installs=auctions_ventana1_reducido.merge(installs,how='left',left_on='device_id',right_on='ref_hash')


In [ ]:
fecha_minima=pd.to_datetime('2019-04-20 00:00:00')
fecha_tope=pd.to_datetime('2019-04-23 00:00:00')


auctions_ventana1_con_installs['tiempo_hasta_instalacion']=(auctions_ventana1_con_installs['created']-fecha_minima)/ np.timedelta64(24, 'h')


auctions_ventana1_reducido=auctions_ventana1_con_installs.groupby(['device_id']).agg({"mean_hour" : min,\
                                                                         "date" : min,\
                                                                         "std_hour" : "min",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min
                                                                        ,'tiempo_hasta_instalacion':"min"})\
                                                                            .reset_index('device_id')

In [ ]:
auctions_ventana1_reducido[['dia_mean','tiempo_hasta_aparicion','dia_desvio_std','tiempo_hasta_instalacion']]=auctions_ventana1_reducido[['dia_mean','tiempo_hasta_aparicion','dia_desvio_std','tiempo_hasta_instalacion']].fillna(np.inf)
auctions_ventana1_reducido['entradas_previas']=auctions_ventana1_reducido['entradas_previas'].fillna(0)
auctions_ventana1_reducido.head()




In [ ]:
fecha_minima=pd.to_datetime('2019-04-21 00:00:00')
fecha_tope=pd.to_datetime('2019-04-24 00:00:00')
auctions_ventana2_con_installs=auctions_ventana2_reducido.merge(installs,how='left',left_on='device_id',right_on='ref_hash')

auctions_ventana2_con_installs['tiempo_hasta_instalacion']=(auctions_ventana2_con_installs['created']-fecha_minima)/ np.timedelta64(24, 'h')


auctions_ventana2_reducido=auctions_ventana2_con_installs.groupby(['device_id']).agg({"mean_hour" : min,\
                                                                         "date" : min,\
                                                                         "std_hour" : "min",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min
                                                                        ,'tiempo_hasta_instalacion':"min"})\
                                                                            .reset_index('device_id')
auctions_ventana2_reducido[['dia_mean','tiempo_hasta_aparicion','dia_desvio_std','tiempo_hasta_instalacion']]=auctions_ventana2_reducido[['dia_mean','tiempo_hasta_aparicion','dia_desvio_std','tiempo_hasta_instalacion']].fillna(np.inf)
auctions_ventana2_reducido['entradas_previas']=auctions_ventana2_reducido['entradas_previas'].fillna(0)
auctions_ventana2_reducido.head()



In [ ]:
fecha_minima=pd.to_datetime('2019-04-22 00:00:00')
fecha_tope=pd.to_datetime('2019-04-25 00:00:00')
auctions_ventana3_con_installs=auctions_ventana3_reducido.merge(installs,how='left',left_on='device_id',right_on='ref_hash')

auctions_ventana3_con_installs['tiempo_hasta_instalacion']=(auctions_ventana3_con_installs['created']-fecha_minima)/ np.timedelta64(24, 'h')


auctions_ventana3_reducido=auctions_ventana3_con_installs.groupby(['device_id']).agg({"mean_hour" : min,\
                                                                         "date" : min,\
                                                                         "mean_std" : "min",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min
                                                                        ,'tiempo_hasta_instalacion':"min"})\
                                                                            .reset_index('device_id')
auctions_ventana3_reducido[['dia_mean','tiempo_hasta_aparicion','dia_desvio_std','tiempo_hasta_instalacion']]=auctions_ventana3_reducido[['dia_mean','tiempo_hasta_aparicion','dia_desvio_std','tiempo_hasta_instalacion']].fillna(np.inf)
auctions_ventana3_reducido['entradas_previas']=auctions_ventana3_reducido['entradas_previas'].fillna(0)
auctions_ventana3_reducido.head()

In [ ]:
fecha_minima=pd.to_datetime('2019-04-23 00:00:00')
fecha_tope=pd.to_datetime('2019-04-26 00:00:00')
auctions_ventana4_con_installs=auctions_ventana4_reducido.merge(installs,how='left',left_on='device_id',right_on='ref_hash')

auctions_ventana4_con_installs['tiempo_hasta_instalacion']=(auctions_ventana4_con_installs['created']-fecha_minima)/ np.timedelta64(24, 'h')


auctions_ventana4_reducido=auctions_ventana3_con_installs.groupby(['device_id']).agg({"mean_hour" : min,\
                                                                         "date" : min,\
                                                                         "std_hour" : "min",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min
                                                                        ,'tiempo_hasta_instalacion':"min"})\
                                                                            .reset_index('device_id')
auctions_ventana4_reducido[['dia_mean','tiempo_hasta_aparicion','dia_desvio_std','tiempo_hasta_instalacion']]=auctions_ventana4_reducido[['dia_mean','tiempo_hasta_aparicion','dia_desvio_std','tiempo_hasta_instalacion']].fillna(np.inf)
auctions_ventana4_reducido['entradas_previas']=auctions_ventana4_reducido['entradas_previas'].fillna(0)
auctions_ventana4_reducido.head()